### IMPORTANDO AS BIBLIOTECAS NECESSARIAS

In [1]:
import requests
import sqlite3
import pandas as pd
import os

### EXTRAINDO OS DADOS DE TODAS AS UNIVERSIDADES PRESENTES

In [2]:
URL = "https://raw.githubusercontent.com/Hipo/university-domains-list/refs/heads/master/world_universities_and_domains.json"
response = requests.get(URL)
if response.status_code == 200:
    universidades_raw = response.json()
    print(f"Total de registros coletados: {len(universidades_raw)}")
else:
    raise Exception(f"Falha ao baixar o JSON: {response.status_code}")


universidades_raw

Total de registros coletados: 10185


[{'name': 'Engineering Institute of Technology',
  'domains': ['student.eit.edu.au'],
  'web_pages': ['https://www.eit.edu.au/'],
  'country': 'Australia',
  'alpha_two_code': 'AU',
  'state-province': None},
 {'name': 'Universitas Nusa Putra',
  'domains': ['nusaputra.ac.id'],
  'web_pages': ['https://nusaputra.ac.id'],
  'country': 'Indonesia',
  'alpha_two_code': 'ID',
  'state-province': None},
 {'name': 'University of Kyrenia',
  'domains': ['std.kyrenia.edu.tr', 'kyrenia.edu.tr'],
  'web_pages': ['https://kyrenia.edu.tr'],
  'country': 'Turkey',
  'alpha_two_code': 'TR',
  'state-province': None},
 {'name': 'Regent University College of Science and Technology',
  'domains': ['regent.edu.gh'],
  'web_pages': ['https://regent.edu.gh'],
  'country': 'Ghana',
  'alpha_two_code': 'GH',
  'state-province': None},
 {'name': 'Wroclaw Akademia Biznesu',
  'domains': ['student.wab.edu.pl', 'wab.edu.pl'],
  'web_pages': ['https://wab.edu.pl'],
  'country': 'Poland',
  'alpha_two_code': 'PL'

### TRANSFORMANDO OS DADOS

In [3]:

def transform_data(universidades):
    return [
        (
            uni.get("name"),
            uni.get("country"),
            uni.get("state-province"),
            ",".join(uni.get("web_pages", [])),
            ",".join(uni.get("domains", []))
        )
        for uni in universidades
    ]

universidades_data = transform_data(universidades_raw)
print(universidades_data[:5]) 


[('Engineering Institute of Technology', 'Australia', None, 'https://www.eit.edu.au/', 'student.eit.edu.au'), ('Universitas Nusa Putra', 'Indonesia', None, 'https://nusaputra.ac.id', 'nusaputra.ac.id'), ('University of Kyrenia', 'Turkey', None, 'https://kyrenia.edu.tr', 'std.kyrenia.edu.tr,kyrenia.edu.tr'), ('Regent University College of Science and Technology', 'Ghana', None, 'https://regent.edu.gh', 'regent.edu.gh'), ('Wroclaw Akademia Biznesu', 'Poland', None, 'https://wab.edu.pl', 'student.wab.edu.pl,wab.edu.pl')]


### CRIANDO O BANCO E INSERINDO OS DADOS

In [ ]:

DB_FILE = "universidades.db"

if os.path.exists(DB_FILE):
    os.remove(DB_FILE)
    print("Banco antigo apagado!")

with sqlite3.connect(DB_FILE) as conn:
    c = conn.cursor()
    
    c.execute('''
        CREATE TABLE universities (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            country TEXT,
            state_province TEXT,
            web_pages TEXT,
            domains TEXT
        )
    ''')
  
    c.executemany("""
        INSERT INTO universities (name, country, state_province, web_pages, domains)
        VALUES (?, ?, ?, ?, ?)
    """, universidades_data)
    
    conn.commit()
    
conn.close()

print("ETL concluído! Banco 'universidades.db' criado e populado com sucesso")





Banco antigo apagado!
✅ ETL concluído! Banco 'universidades.db' criado e populado com sucesso.


### CRIANDO AS CONSULTAS

In [12]:
with sqlite3.connect(DB_FILE) as conn:
    df_total_por_pais = pd.read_sql_query("""
        SELECT country, COUNT(*) AS total
        FROM universities
        GROUP BY country
        ORDER BY total DESC
    """, conn)
    display(df_total_por_pais)

    df_brasil = pd.read_sql_query("""
        SELECT name
        FROM universities
        WHERE country = 'Brazil'
    """, conn)
    display(df_brasil)

    df_tech = pd.read_sql_query("""
        SELECT name, country
        FROM universities
        WHERE name LIKE '%Technology%'
    """, conn)
    display(df_tech)

    df_pernambuco = pd.read_sql_query("""
        SELECT name, country
        FROM universities
        WHERE name LIKE '%Pernambuco%'
    """, conn)
    display(df_pernambuco)
   


,country,total
0,United States,2348
1,Japan,572
2,India,473
3,China,397
4,Germany,318
...,...,...
197,Bhutan,1
198,Bermuda,1
199,Barbados,1
200,Bahamas,1


,name
0,Universidade Comunitária da Região de Chapecó ...
1,"Centro Universitário de Brasília, UNICEUB"
2,Centro Universitário Barao de Maua
3,Universidade Braz Cubas
4,Universidade Candido Mendes
...,...
185,Centro Universitário Álvares Penteado - FECAP
186,Centro Universitário Facens - UniFACENS
187,Centro Universitário Antônio Eufrásio de Toled...
188,Faculdade Impacta - IMPACTA


,name,country
0,Engineering Institute of Technology,Australia
1,Regent University College of Science and Techn...,Ghana
2,Bhagwan Parshuram Institute of Technology,India
3,Indian Institute of Information Technology Sri...,India
4,DAV Institute of Engineering & Technology,India
...,...,...
555,Dr. Sudhir Chandra Sur Institute of Technology...,India
556,Institute of Technology and Business (VŠTE),Czech Republic
557,Kalinga Institute of Industrial Technology (KIIT),India
558,Helena Chodkowska University of Technology and...,Poland


,name,country
0,Universidade Federal de Pernambuco,Brazil
1,Universidade Federal Rural de Pernambuco,Brazil
2,Universidade Católica de Pernambuco,Brazil
3,Universidade de Pernambuco,Brazil
4,Polytechnic University of Pernambuco,Brazil
